In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
from torch import nn
from torch.utils import data

In [ ]:
# 以下为给琪琪的接口

In [2]:
pca_per_ppd = pd.read_csv('./pca_per.csv',index_col=0,header=None)
pca_job_ppd = pd.read_csv('./pca_job.csv',index_col=0,header=None)

In [3]:
pca_per_myd = pd.read_csv('./pca_per_myd.csv',index_col=0,header=None)
pca_job_myd = pd.read_csv('./pca_job_myd.csv',index_col=0,header=None)

In [4]:
def get_input_ppd(pid,rid):
  per = pca_per_ppd.loc[pid].values
  job = pca_job_myd.loc[rid].values
  all = np.concatenate([per,job],axis=0)
  return all

def get_input_myd(pid,rid):
  per = pca_per_myd.loc[pid].values
  job = pca_job_myd.loc[rid].values
  all = np.concatenate([per,job],axis=0)
  return all

In [5]:
net_ppd = torch.load('./Sigmoid-200.pt')
net_myd = torch.load('./Sigmoid-MYD.pt')

In [6]:
# 硬核条件，这里的序号是0~N-1
hd_job = pd.read_csv('./hardcore_position.csv',index_col=0,header=0,encoding='gbk')
hd_job.index = [i for i in range(hd_job.shape[0])]
hd_per = pd.read_csv('./hardcore_user.csv',index_col=0,header=0,encoding='gbk')
hd_per.index = [i for i in range(hd_per.shape[0])]
hd_job.shape, hd_per.shape

((1575, 4), (10877, 3))

In [7]:
def is_fw(pid):
  val = fw_list.loc[pid,'是否为废人']
  return val==1

fw_list = pd.read_csv('./fw.txt',sep=' ',index_col=0)

In [11]:
# 属性嵌入，这里的序号是0~N-1
# 用于拼接向量 and 判断行业
a_emb_job = pd.read_csv('./attribute_embedding_position.txt',sep=' ',index_col=0,header=None,na_values=[])
a_emb_per = pd.read_csv('./attribute_embedding_user.txt',sep=' ',index_col=0,header=None,na_values=[])

In [8]:
def judge_hangye(pid,rid): # 0~N-1
  # print(pid,rid)
  x = a_emb_per.loc[pid,6:14].values.astype('int')
  y = a_emb_job.loc[rid,6:14].values.astype('int')
  sum = (x*y).sum()
  return sum > 0

def judge_gongzi(pid,rid): # 0~N-1
  p2, p1 = hd_per.iloc[pid,:-1]
  r2, r1 = hd_job.iloc[rid,-2:]
  return max(p1,r1)<=min(p2,r2)

def judge_type(pid,rid):
  pt = hd_per.iloc[pid,-1]
  rt = hd_job.iloc[rid,0]
  # print(pt,rt)
  return pt == rt

def judge(pid,rid):
  # b1 = judge_hangye(pid,rid)
  b2 = judge_gongzi(pid,rid)
  b3 = judge_type(pid,rid)
  return b2 and b3

In [13]:
judge(3108,365)

False

In [19]:
ppd_buf = {}
myd_buf = {}
for i in pca_per_ppd.index:
  ppd_buf[i] = np.zeros(1575)-1
  myd_buf[i] = np.zeros(1575)-1

def get_ppd(pid,rid):
  if judge(pid,rid)==False: # 硬性条件不满足为0
    return 0
  elif is_fw(pid):  # 废物为1
    return 0.1
  elif ppd_buf[pid][rid]>=0:
    return ppd_buf[pid][rid]
  else:
    ppd = net_ppd(torch.Tensor(get_input_ppd(pid,rid))).item()
    print(ppd)
    # print(f'ppd({pid},{rid})={ppd}')
  ppd = min(ppd,10)
  ppd = max(ppd,1)/10
  ppd_buf[pid][rid] = ppd
  return ppd

def get_myd(pid,rid):
  if judge(pid,rid)==False: # 硬性条件不满足为0
    return 0
  elif is_fw(pid):  # 废物为1
    return 0.1
  elif myd_buf[pid][rid]>=0:
    return myd_buf[pid][rid]
  else:
    myd = net_myd(torch.Tensor(get_input_myd(pid,rid))).item()
    print(myd)
    # print(f'ppd({pid},{rid})={ppd}')
  myd = min(myd,10)
  myd = max(myd,1)/10
  myd_buf[pid][rid] = myd
  return myd

In [20]:
# 调用实例，使用时可以把函数里的print关掉
# 仅第一次调用神经网络，即建立缓存时会print
a,b = 317,46
get_ppd(a,b),get_myd(a,b)

(0, 0)

In [17]:
judge(a,b)

False